In [31]:
import math
import os
import pandas as pd
import shutil
import numpy as np

In [ ]:
ASSETS_PATH = ""

DATA_YEAR_VALUE = ""
REPORT_YEAR_VALUE = ""
MONTH_VALUE = ""

INTRODUCTION_TITEL = ""
INTRODUCTION_TEXT = ""

PERFORMANCE_TABLE_PATH = ""

In [32]:
# tabelle-bausteine

document_frame = r"""
\documentclass[11pt]{article}
\usepackage[german]{babel}
\usepackage[utf8]{inputenc}
\usepackage[export]{adjustbox}
\usepackage{booktabs}
\usepackage{fancyhdr}
\usepackage{fontspec}
\usepackage[T1]{fontenc}
\usepackage{graphicx}
\usepackage{lastpage}
\usepackage[onehalfspacing]{setspace}
\usepackage{multicol}
\usepackage{multirow}
\usepackage{tabularx}
\usepackage{tikz}
\usetikzlibrary{calc}
\usepackage{verbatim}
\usepackage{xcolor}
\usepackage[headheight=2.25cm, left=3cm,right=2cm,top=3.5cm,bottom=2cm]{geometry}
\setmainfont{Arial}


\begin{document}
\begin{titlepage}
    \begin{tikzpicture}[overlay, remember picture]
    \node[anchor=north west, %anchor is upper left corner of the graphic
            xshift=-14.61cm, %shifting around
            yshift=-10.8cm] 
            at (current page.north west) %left upper corner of the page
            {\includegraphics[width=42cm]{KreisGrün}}; 
    \node[anchor=north west, %anchor is upper left corner of the graphic
            xshift=15.98cm, %shifting around
            yshift=-25.07cm] 
            at (current page.north west) %left upper corner of the page
            {\includegraphics[scale=.35]{PHSGlogow}}; 
    \end{tikzpicture}

    \begin{tikzpicture}[overlay, remember picture]
    \node[anchor=north west,
            xshift=3cm,
            yshift=-15cm,
            align=left,
            text width=14cm] 
            at (current page.north west)
            {{\color{white} \fontsize{35}{11}\selectfont\baselineskip=40pt Leistungsbereich __area__ \par
            \vspace{1cm}
            \color{white} \fontsize{18}{11}\selectfont Report zu den Qualitätscockpits __data_year__ \par
            \vspace{.5cm}
            \color{white} \fontsize{12.5}{11}\selectfont HEQM \par}};
    \end{tikzpicture}
\end{titlepage}



\pagestyle{fancy}
	\fancyhf{}
	\renewcommand{\headrulewidth}{0pt}
	\rhead{\includegraphics[scale=.45]{PHSGlogoGrün}}
	\fancyfoot[L]{\fontsize{9}{11} \selectfont HEQM}
	\fancyfoot[C]{\fontsize{9}{11} \selectfont __month__ __report_year__}
	\fancyfoot[R]{\fontsize{9}{11} \selectfont Seite {\thepage}\ von {\pageref{LastPage}}}

\section{__introduction_titel__}
\label{sec: einleitung}
__introduction_text__


__content__

\end{document}
"""

top_table_head_template = r"""
\newpage
\renewcommand\tabularxcolumn[1]{m{#1}}
\renewcommand{\arraystretch}{1.5}
\begin{tabularx}{\textwidth}{>{\raggedright \arraybackslash}Xc}
    \toprule
    Indikator & Performance Leistungsbereich\\
    \midrule
"""

top_table_content_template = r"""
    \noindent\parbox[c]{\hsize}{__indicator_text__} & \includegraphics[valign={M}, scale=0.35]{__area_mean__}\\
"""

top_table_end_template = r"""
    \bottomrule
\end{tabularx}
"""

bottom_table_head_template = r"""
\begin{tabularx}{\textwidth}{>{\raggedright}Xrc}
    \toprule
    Organisationseinheiten & Erfüllungsgrad & Status\\
    \midrule
"""

bottom_table_content_template = r"""
    __department__ & __dep_performance__ \% & \includegraphics[valign={m}, scale=0.11]{__dept_mean__}\\
"""

bottom_table_end_template = r"""
    \bottomrule
\end{tabularx}
"""

In [8]:
# bilder zum einsetzten

circle_green_path = ASSETS_PATH +  "KreisGrün.png"
circle_blue_path = ASSETS_PATH +  "KreisBlau.png"
logo_green_path = ASSETS_PATH +  "PHSGlogoGrün.png"
logo_blue_path = ASSETS_PATH +  "PHSGlogoBlau.png"
logo_white_path = ASSETS_PATH +  "PHSGlogoWeiss.png"

dot_green_path = ASSETS_PATH +  "dotGrün.png"
dot_red_path = ASSETS_PATH +  "dotRot.png"
dot_grey_path = ASSETS_PATH +  "dotGrau.png"

lvl0_path = ASSETS_PATH +  "lvl0.png"
lvl0_path = ASSETS_PATH +  "lvl0.png"
lvl1_path = ASSETS_PATH +  "lvl1.png"
lvl2_path = ASSETS_PATH +  "lvl2.png"
lvl3_path = ASSETS_PATH +  "lvl3.png"
lvl4_path = ASSETS_PATH +  "lvl4.png"
lvl5_path = ASSETS_PATH +  "lvl5.png"
lvl6_path = ASSETS_PATH +  "lvl6.png"
lvl7_path = ASSETS_PATH +  "lvl7.png"
lvl8_path = ASSETS_PATH +  "lvl8.png"
lvl9_path = ASSETS_PATH +  "lvl9.png"
lvl10_path = ASSETS_PATH +  "lvl10.png"

sub_lvl0_path = ASSETS_PATH +  "slvl0.png"
sub_lvl1_path = ASSETS_PATH +  "slvl1.png"
sub_lvl2_path = ASSETS_PATH +  "slvl2.png"
sub_lvl3_path = ASSETS_PATH +  "slvl3.png"
sub_lvl4_path = ASSETS_PATH +  "slvl4.png"
sub_lvl5_path = ASSETS_PATH +  "slvl5.png"
sub_lvl6_path = ASSETS_PATH +  "slvl6.png"
sub_lvl7_path = ASSETS_PATH +  "slvl7.png"
sub_lvl8_path = ASSETS_PATH +  "slvl8.png"
sub_lvl9_path = ASSETS_PATH +  "slvl9.png"
sub_lvl10_path = ASSETS_PATH +  "slvl10.png"

no_data = "--"

In [34]:
report_date = {"data_year": DATA_YEAR_VALUE, "report_year": REPORT_YEAR_VALUE, "month": MONTH_VALUE}

areas = ["Ausbildung", "Forschung & Entwicklung", "Weiterbildung und Dienstleistung"]

areas_abbr = {
    "Ausbildung" : "A",
    "Forschung & Entwicklung" : "FE",
    "Weiterbildung und Dienstleistung" : "WB",
}

departments = [
    "Institut Berufsbildung",
    "Institut Bewegung, Sport und Gesundheit",
    "Institut Digitale und Informatische Bildung",
    "Institut Frühe Bildung 0 bis 8",
    "Institut Gesellschaftswissenschaftliche Bildung",
    "Institut Kulturelle und Ästhetische Bildung",
    "Institut Mathematische, Naturwissenschaftliche und Technische Bildung",
    "Institut Pädagogische Psychologie",
    "Institut Schule und Profession",
    "Institut Sprachliche und Literarische Bildung",
    "Zentrum Berufspraktische Studien",
    "Zentrum Weiterbildung",
    "Regionale Didaktische Zentren",
    "Zentrum Digitalisierung und Bildung",
    "Zentrum Internationalisierung, Mobilität und Aussenbeziehungen",
    "Zentrum Wissenschaftsberatung",
    "Zentrum Nachhaltige Entwicklung",
]

subjects = {
    "Institut Berufsbildung": [],
    "Institut Bewegung, Sport und Gesundheit": [
        "FB Bewegung & Sport",
        "FB Gesundheitsbildung",
    ],
    "Institut Digitale und Informatische Bildung": [],
    "Institut Frühe Bildung 0 bis 8": [
        "TB Familienbildung & Partizipation",
        "TB Frühe, fachliche Bildung und Entwicklungsorientierung",
        "TB Multiprofessionale Kooperation und Inklusion",
        "TB Spiel",
    ],
    "Institut Gesellschaftswissenschaftliche Bildung": ["FB ERG", "FB RZG", "FB WAH"],
    "Institut Kulturelle und Ästhetische Bildung": [
        "FB Bildnerisches Gestalten",
        "FB Instrumentalunterricht",
        "FB Schulmusik",
        "FB Technisches Gestalten",
        "FB Textiles Gestalten",
        "TB Darstellende Künste",
    ],
    "Institut Mathematische, Naturwissenschaftliche und Technische Bildung": [
        "FB Mathematik",
        "FB Natur, Mensch, Gesellschaft & Technik",
    ],
    "Institut Pädagogische Psychologie": [
        "TB Individuelle Förderung und Differenzierung",
        "TB Pädagogisch-psychologische Diagnostik",
    ],
    "Institut Schule und Profession": [
        "TB Didaktik: Lehr- und Lernarrangements",
        "TB Lehrperson und Klasse: Unterricht und soziale Interaktion",
        "TB Schule: Profession und Kooperation",
    ],
    "Institut Sprachliche und Literarische Bildung": [
        "FB Deutsch",
        "FB Englisch",
        "FB Französisch",
        "FB Italienisch und Latein",
    ],
    "Zentrum Berufspraktische Studien": [],
    "Zentrum Weiterbildung": [],
    "Regionale Didaktische Zentren": [],
    "Zentrum Digitalisierung und Bildung": [],
    "Zentrum Internationalisierung, Mobilität und Aussenbeziehungen": [],
    "Zentrum Wissenschaftsberatung": [],
    "Zentrum Nachhaltige Entwicklung": [],
}

In [35]:
path_to_performance_table = PERFORMANCE_TABLE_PATH
df_report_data = pd.read_csv(path_to_performance_table)
df_report_data = df_report_data.sort_values(by=["report_level"], ascending=False)

report_path = "ReportLeistungsbereich"

if os.path.isdir(report_path) == False:
    os.mkdir(report_path)


In [36]:
for area in areas:
    document = document_frame
    document = document.replace("__area__", area)
    document = document.replace("__data_year__", report_date["data_year"])
    document = document.replace("__report_year__", report_date["report_year"])
    document = document.replace("__month__", report_date["month"])
    document = document.replace("__introduction_titel__", INTRODUCTION_TITEL)
    document = document.replace("__introduction_text__", INTRODUCTION_TEXT)

    filter_str = areas_abbr[area]
    df_report_data_filtered = df_report_data[df_report_data["indicator_id"].str.contains(filter_str)]

    content = ""

    for indicator in df_report_data_filtered["indicator_id"]:

        table = ""
        top_table_content = top_table_content_template
        bottom_table_content = bottom_table_content_template

        i = df_report_data_filtered.isin([indicator]).any(axis=1).idxmax() # get index of current indicator

        if df_report_data_filtered["report_level"][i] == "Leistungsbereich":
            indicator_text = str(df_report_data_filtered["indicator_text"][i])
            indicator_text = indicator_text.replace("&", "\&")
            indicator_text = indicator_text.replace("%", "\%")
            area_performance = str(df_report_data_filtered[f"{area}_performance"][i])

            top_table_content = top_table_content.replace("__indicator_text__", indicator_text)
            if area_performance == "True":
                top_table_content = top_table_content.replace("__area_mean__", dot_green_path)
            elif area_performance == "False":
                top_table_content = top_table_content.replace("__area_mean__", dot_red_path)
            else:
                top_table_content = top_table_content.replace("__area_mean__", dot_grey_path)
            
            table += top_table_head_template
            table += top_table_content
            table += top_table_end_template
            content += table
        
        elif df_report_data_filtered["report_level"][i] == "nan":
            raise KeyError(f"Report-Level for {df_report_data_filtered['indicator_id'][i]} is not defiend.")

        else:
            ## top table
            indicator_text = str(df_report_data_filtered["indicator_text"][i])
            indicator_text = indicator_text.replace("&", "\&")
            indicator_text = indicator_text.replace("%", "\%")

            top_table_content = top_table_content.replace("__indicator_text__", indicator_text)
            # __area_mean__ wird weiter unten berechnet, da es die Performance aus den departments braucht

            table += top_table_head_template
            table += top_table_content
            table += top_table_end_template
            table += bottom_table_head_template


            ## bottom table
            department_performances = []

            for department in departments:
                df_department = pd.Series(dtype=bool)
                bottom_table_content = bottom_table_content_template
                count_true = 0
                count_false = 0

                # TODO make check that there is never subjects and department performance in the data for the same indicator at the same time (row)
                # get subject level performance
                for key, values in subjects.items():
                    if department == key:
                        for value in values:
                            search_string = f"{value}_performance"
                            if search_string in df_report_data_filtered.columns:
                                df_department[f"{value}_performance"] = df_report_data_filtered.loc[i, f"{value}_performance"]

                # get department level performance
                search_string = f"{department}_performance"
                if search_string in df_report_data_filtered.columns:
                    df_department[f"{department}_performance"] = df_report_data_filtered.loc[i, f"{department}_performance"]
                        
                if len(df_department) == 0:
                    department_performance = np.nan

                if len(df_department) >= 0:
                    department_performance = df_department.astype(float).mean() * 100
                else:
                    department_performance = np.nan

                    # Best mean function ever but too good
                    #for _, row in df_department.iterrows():
                    #    print(row)
                    #    performance_list = row.tolist()
                    #    count_true = performance_list.count(True)
                    #    count_false = performance_list.count(False)
                    #    if (count_true + count_false) > 0: # de if isch dinne, weil es dividion by zero geh het
                    #        department_performance = round(count_true / (count_true + count_false) * 100, 2)
                    #    else:
                    #        department_performance = np.nan
                
                bottom_table_content = bottom_table_content.replace("__department__", department)
                bottom_table_content = bottom_table_content.replace("__dep_performance__", str(department_performance))
                bottom_table_content = bottom_table_content.replace("nan", no_data)

                if pd.isna(department_performance):
                    bottom_table_content = bottom_table_content.replace("__dept_mean__", dot_grey_path)
                elif department_performance < 10:
                    bottom_table_content = bottom_table_content.replace("__dept_mean__", sub_lvl0_path)
                elif department_performance < 20:
                    bottom_table_content = bottom_table_content.replace("__dept_mean__", sub_lvl1_path)
                elif department_performance < 30:
                    bottom_table_content = bottom_table_content.replace("__dept_mean__", sub_lvl2_path)
                elif department_performance < 40:
                    bottom_table_content = bottom_table_content.replace("__dept_mean__", sub_lvl3_path)
                elif department_performance < 50:
                    bottom_table_content = bottom_table_content.replace("__dept_mean__", sub_lvl4_path)
                elif department_performance < 60:
                    bottom_table_content = bottom_table_content.replace("__dept_mean__", sub_lvl5_path)
                elif department_performance < 70:
                    bottom_table_content = bottom_table_content.replace("__dept_mean__", sub_lvl6_path)
                elif department_performance < 80:
                    bottom_table_content = bottom_table_content.replace("__dept_mean__", sub_lvl7_path)
                elif department_performance < 90:
                    bottom_table_content = bottom_table_content.replace("__dept_mean__", sub_lvl8_path)
                elif department_performance < 100:
                    bottom_table_content = bottom_table_content.replace("__dept_mean__", sub_lvl9_path)
                elif department_performance == 100:
                    bottom_table_content = bottom_table_content.replace("__dept_mean__", sub_lvl10_path)
                else:
                    raise ValueError(f"'department_performance' has unexpected value: {department_performance}")
                    
                table += bottom_table_content

                department_performances.append(department_performance)
                department_performances = list(filter(lambda x: not math.isnan(x), department_performances))
            
            table += bottom_table_end_template

            if len(department_performances) == 0:
                area_performance = float("nan")
            if len(department_performances) >= 0:
                area_performance = sum(department_performances) / len(departments)
            else:
                raise ValueError(f"List 'department_performances' has unexpected form: {department_performances}")
            
            if pd.isna(area_performance):
                table = table.replace("__area_mean__", dot_grey_path)
            elif area_performance < 10:
                table = table.replace("__area_mean__", lvl0_path)
            elif area_performance < 20:
                table = table.replace("__area_mean__", lvl1_path)
            elif area_performance < 30:
                table = table.replace("__area_mean__", lvl2_path)
            elif area_performance < 40:
                table = table.replace("__area_mean__", lvl3_path)
            elif area_performance < 50:
                table = table.replace("__area_mean__", lvl4_path)
            elif area_performance < 60:
                table = table.replace("__area_mean__", lvl5_path)
            elif area_performance < 70:
                table = table.replace("__area_mean__", lvl6_path)
            elif area_performance < 80:
                table = table.replace("__area_mean__", lvl7_path)
            elif area_performance < 90:
                table = table.replace("__area_mean__", lvl8_path)
            elif area_performance < 100:
                table = table.replace("__area_mean__", lvl9_path)
            elif area_performance == 100:
                table = table.replace("__area_mean__", lvl10_path)
            else:
                raise ValueError(f"'area_performance' has unexpected value: {area_performance}")

            content += table

    report = document.replace("__content__", content)

    report_name = f"{area}{report_date['data_year']}.tex"
    report_name = report_name.replace("&", "")
    report_name = report_name.replace(" ", "")

    with open(f"{report_path}/{report_name}", "w+") as f:
        f.writelines(report)

    shutil.copyfile(circle_green_path, f"{report_path}/KreisGrün.png")
    shutil.copyfile(logo_green_path, f"{report_path}/PHSGlogoGrün.png")
    shutil.copyfile(logo_white_path, f"{report_path}/PHSGlogow.png")



    os.chdir(f"{report_path}/")
    os.system(f"xelatex -synctex=1 -interaction=nonstopmode {report_name}")
    os.system(f"xelatex -synctex=1 -interaction=nonstopmode {report_name}")
    os.chdir("..")

This is XeTeX, Version 3.141592653-2.6-0.999994 (TeX Live 2022) (preloaded format=xelatex)
 restricted \write18 enabled.
entering extended mode
(./Ausbildung2022.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-02-24>
(/usr/local/texlive/2022/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(/usr/local/texlive/2022/texmf-dist/tex/latex/base/size11.clo))
(/usr/local/texlive/2022/texmf-dist/tex/generic/babel/babel.sty
(/usr/local/texlive/2022/texmf-dist/tex/generic/babel/xebabel.def
(/usr/local/texlive/2022/texmf-dist/tex/generic/babel/txtbabel.def))
(/usr/local/texlive/2022/texmf-dist/tex/generic/babel-german/german.ldf
(/usr/local/texlive/2022/texmf-dist/tex/generic/babel-german/germanb.ldf)))
(/usr/local/texlive/2022/texmf-dist/tex/latex/base/inputenc.sty

Package inputenc Warning: inputenc package ignored with utf8 based engines.

) (/usr/local/texlive/2022/texmf-dist/tex/latex/adjustbox/adjustbox.sty
(/usr/